# Create VIEWs on SEC DERA data by LEI

## Full-year FY REVENUES

## Point-in-time FLOAT, DEBT, CASH, ASSETS, (EV = FLOAT + DEBT - CASH, EVIC = FLOAT + DEBT)

In [42]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [43]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

import pandas as pd

In [44]:
from osc_ingest_trino import *

trino_bucket = attach_s3_bucket("S3_DEV")

### Revenue

In [113]:
qres = engine.execute("""
create or replace view sec_dera.fy_revenue_by_lei as
select name, lei, tname, fy, N.ddate, sic, max(value) as revenue
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate
     left join sec_dera.ticker T on S.cik=T.cik
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and coreg is NULL
and qtrs=4
and uom='USD'
and (N.tag='Revenues'
     or N.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
     or N.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
     or N.tag='RevenuesNetOfInterestExpense'
     or N.tag='RegulatedAndUnregulatedOperatingRevenue'
     or N.tag='RegulatedOperatingRevenuePipelines')
group by name, lei, tname, fy, N.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.fy_revenue_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.fy_revenue_by_lei where fy>=DATE('2020-01-01') order by fy, LEI limit 20
""")
display(qres.fetchall())

[(True,)]

[(11317,)]

[('WESTINGHOUSE AIR BRAKE TECHNOLOGIES CORP', '06BTX5UWZD0GQ5N5Y745', 'wab', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 3743, 7556100000.0),
 ('WEYERHAEUSER CO', '08IRJODWFYBI7QWRGS31', 'wy', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 6798, 7532000000.0),
 ('BALL CORP', '0BGI85ALH27ZJP15DY16', 'bll', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 3411, 11781000000.0),
 ('BLACK STONE MINERALS, L.P.', '0C0OFK66613BCRQJ7F34', 'bsm', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 1311, 342751000.0),
 ('AECOM', '0EEB8GF0W0NPCIHZX097', 'acm', '2020-01-01 00:00:00.000', '2020-09-30 00:00:00.000', 8711, 13239976000.0),
 ('SP PLUS CORP', '0IDE18EMH1CUKQCUYE69', 'sp', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 7510, 1086900000.0),
 ('SP PLUS CORP', '0IDE18EMH1CUKQCUYE69', 'crlkp', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 7510, 1086900000.0),
 ('CHARTER COMMUNICATIONS, INC.', '0J0XRGZE3PBRFEZ7MV65', 'chtr', '2020-01-01 00:00:00.000', '20

### Float (market cap)

In [112]:
qres = engine.execute("""
create or replace view sec_dera.float_by_adsh_ddate as
select S.adsh, fy, max(ddate) as f_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A'  or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and (tag='EntityPublicFloat'
             or tag='EntitysPublicFloat'
             or tag='FreeFloat'
             or tag='PublicFloat'
             or tag='PublicFloatValue')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.float_by_lei as
select name, lei, tname, S.fy, N.ddate, sic, max(value) as market_cap
from sec_dera.sub as S
     join sec_dera.float_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.f_ddate=N.ddate
     left join sec_dera.ticker T on S.cik=T.cik
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and value>0
and coreg is NULL
and qtrs=0
and uom='USD'
and (tag='EntityPublicFloat'
     or tag='EntitysPublicFloat'
     or tag='FreeFloat'
     or tag='PublicFloat'
     or tag='PublicFloatValue')
group by name, lei, tname, S.fy, N.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.float_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.float_by_lei where fy>=DATE('2020-01-01') order by fy, LEI limit 20
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(10975,)]

[('WESTINGHOUSE AIR BRAKE TECHNOLOGIES CORP', '06BTX5UWZD0GQ5N5Y745', 'wab', '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 3743, 10500000000.0),
 ('WEYERHAEUSER CO', '08IRJODWFYBI7QWRGS31', 'wy', '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 6798, 16800000000.0),
 ('BALL CORP', '0BGI85ALH27ZJP15DY16', 'bll', '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 3411, 22700000000.0),
 ('BLACK STONE MINERALS, L.P.', '0C0OFK66613BCRQJ7F34', 'bsm', '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 1311, 1015788702.0),
 ('AECOM', '0EEB8GF0W0NPCIHZX097', 'acm', '2020-01-01 00:00:00.000', '2020-03-31 00:00:00.000', 8711, 4500000000.0),
 ('SP PLUS CORP', '0IDE18EMH1CUKQCUYE69', 'crlkp', '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 7510, 476800000.0),
 ('SP PLUS CORP', '0IDE18EMH1CUKQCUYE69', 'sp', '2020-01-01 00:00:00.000', '2020-06-30 00:00:00.000', 7510, 476800000.0),
 ('CHARTER COMMUNICATIONS, INC.', '0J0XRGZE3PBRFEZ7MV65', 'chtr', '2020-01-01 00:00:00.000', '20

### Cash

In [110]:
qres = engine.execute("""
create or replace view sec_dera.cash_by_adsh_ddate as
select S.adsh, fy, max(ddate) as c_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A'  or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and (tag='CashAndCashEquivalentsAtCarryingValue' or tag='Cash' or tag='CashEquivalentsAtCarryingValue' or tag='CashAndCashEquivalents')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.cash_by_lei as
select name, lei, S.fy, N.ddate, sic, max(value) as cash
from sec_dera.sub as S
     join sec_dera.cash_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.c_ddate=N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and coreg is NULL
and qtrs=0
and uom='USD'
and (tag='CashAndCashEquivalentsAtCarryingValue' or tag='Cash' or tag='CashEquivalentsAtCarryingValue' or tag='CashAndCashEquivalents')
group by name, lei, S.fy, N.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.cash_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.cash_by_lei where fy>=DATE('2020-01-01') order by fy, LEI limit 20
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(11349,)]

[('WESTINGHOUSE AIR BRAKE TECHNOLOGIES CORP', '06BTX5UWZD0GQ5N5Y745', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 3743, 598700000.0),
 ('WEYERHAEUSER CO', '08IRJODWFYBI7QWRGS31', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 6798, 495000000.0),
 ('BALL CORP', '0BGI85ALH27ZJP15DY16', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 3411, 1366000000.0),
 ('BLACK STONE MINERALS, L.P.', '0C0OFK66613BCRQJ7F34', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 1311, 1796000.0),
 ('AECOM', '0EEB8GF0W0NPCIHZX097', '2020-01-01 00:00:00.000', '2020-09-30 00:00:00.000', 8711, 1708332000.0),
 ('SP PLUS CORP', '0IDE18EMH1CUKQCUYE69', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 7510, 13900000.0),
 ('CHARTER COMMUNICATIONS, INC.', '0J0XRGZE3PBRFEZ7MV65', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 4841, 998000000.0),
 ('MDU RESOURCES GROUP INC', '0T6SBMK3JTBI1JR36794', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 1400, 59547000.0),
 ('REALNETWO

### Debt

In [109]:
# Reiterating comment embedded below: we should sum 'LongTermDebtNoncurrent' and 'LongTermDebtCurrent' as one term we MAX with others
qres = engine.execute("""
create or replace view sec_dera.debt_by_adsh_ddate as
select S.adsh, fy, max(ddate) as d_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A'  or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
             or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
             or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
             or tag='OperatingLeaseLiabilityNoncurrent'
             or tag='SubordinatedDebt' or tag='ConvertibleDebt'
             or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
             -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
             or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.debt_by_lei as
select name, lei, S.fy, N.ddate, sic, max(value) as debt
from sec_dera.sub as S
     join sec_dera.debt_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A' or S.form='20-F')
and coreg is NULL
and qtrs=0
and uom='USD'
and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
     or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
     or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
     or tag='OperatingLeaseLiabilityNoncurrent'
     or tag='SubordinatedDebt' or tag='ConvertibleDebt'
     or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
     -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
     or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by name, lei, S.fy, N.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.debt_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.debt_by_lei where fy>=DATE('2020-01-01') order by fy, LEI limit 20
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(8803,)]

[('WESTINGHOUSE AIR BRAKE TECHNOLOGIES CORP', '06BTX5UWZD0GQ5N5Y745', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 3743, 4239400000.0),
 ('WEYERHAEUSER CO', '08IRJODWFYBI7QWRGS31', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 6798, 6943000000.0),
 ('BALL CORP', '0BGI85ALH27ZJP15DY16', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 3411, 7783000000.0),
 ('BLACK STONE MINERALS, L.P.', '0C0OFK66613BCRQJ7F34', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 1311, 121000000.0),
 ('AECOM', '0EEB8GF0W0NPCIHZX097', '2020-01-01 00:00:00.000', '2020-09-30 00:00:00.000', 8711, 2085000000.0),
 ('SP PLUS CORP', '0IDE18EMH1CUKQCUYE69', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 7510, 337100000.0),
 ('CHARTER COMMUNICATIONS, INC.', '0J0XRGZE3PBRFEZ7MV65', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 4841, 82752000000.0),
 ('MDU RESOURCES GROUP INC', '0T6SBMK3JTBI1JR36794', '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 1400, 2213130000.0),
 ('

### Assets

In [107]:
qres = engine.execute("""
create or replace view sec_dera.assets_by_adsh_ddate as
select S.adsh, fy, max(ddate) as a_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and N.ddate>=S.fy and date_add('year', 1, S.fy)>N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A'  or S.form='20-F')
        and coreg is NULL
        and qtrs=0
        and uom='USD'
        and tag='Assets'
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.assets_by_lei as
select name, lei, S.fy, N.ddate, sic, value as assets
from sec_dera.sub as S
     join sec_dera.assets_by_adsh_ddate as AD on S.adsh=AD.adsh and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.a_ddate=N.ddate
where S.fp='FY' and (S.form='10-K' or S.form='10-K/A'  or S.form='20-F')
and coreg is NULL
and qtrs=0
and uom='USD'
and tag='Assets'
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.assets_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.assets_by_lei where ddate>=DATE('2020-01-01') order by LEI, ddate limit 20
""")
display(qres.fetchall())

In [115]:
qres = engine.execute("""
create or replace view sec_dera.financials_by_lei as
select R.name, R.lei, R.tname, R.fy, R.sic, revenue, cash, debt, assets, market_cap
from sec_dera.fy_revenue_by_lei as R
     join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
display(qres.fetchall())

qres = engine.execute("""
select count (*) from sec_dera.financials_by_lei
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.financials_by_lei where R.fy>=DATE('2020-01-01') order by LEI limit 20 limit 10
""")
display(qres.fetchall())

[(True,)]

TrinoQueryError: TrinoQueryError(type=INTERNAL_ERROR, name=REMOTE_TASK_MISMATCH, message="Could not communicate with the remote task. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (10.131.3.46:8080)", query_id=20211213_210432_00227_85hmd)

In [118]:
qres = engine.execute("""
explain analyze
select R.name, R.lei, R.tname, R.fy, R.sic, revenue, cash -- , debt --, assets, market_cap
from sec_dera.fy_revenue_by_lei as R
     join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     -- join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     -- join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     -- join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
l = qres.fetchall()

TrinoQueryError: TrinoQueryError(type=INTERNAL_ERROR, name=TOO_MANY_REQUESTS_FAILED, message="Encountered too many errors talking to a worker node. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (updating task http://10.129.2.21:8080/v1/task/20211213_211544_00230_85hmd.11.2 - 140 failures, failure duration 300.54s, total failed request time 234.24s)", query_id=20211213_211544_00230_85hmd)

In [ ]:
qres = engine.execute("""
explain analyze
select R.name, R.lei, R.tname, R.fy, R.sic, revenue,
    -- cash,
    debt,
    -- assets,
    market_cap
from sec_dera.fy_revenue_by_lei as R
     -- join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     -- join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
l = qres.fetchall()

In [13]:
qres = engine.execute("""
create or replace view sec_dera.financials_by_lei as
select name, lei, fy, Nrevenue.ddate as r_ddate, sic,
       max(Nrevenue.value) as revenue,
       max(Ncash.value) as cash,
       max(Ndebt.value) as debt,
       max(Nassets.value) as assets,
       max(Nfloat.value) as market_cap
       -- max(Nfloat.value)+max(Ndebt.value)-max(Ncash.value) as ev,
       -- max(Nfloat.value)+max(Ndebt.value) as evic
from sec_dera.sub as S
      join sec_dera.num as Nrevenue on S.adsh=Nrevenue.adsh
            and Nrevenue.qtrs=4
            and Nrevenue.ddate>=S.fy and date_add('year', 1, S.fy)>Nrevenue.ddate
      join sec_dera.num as Nassets on S.adsh=Nassets.adsh and Nassets.ddate=Nrevenue.ddate
      join sec_dera.num as Ndebt on S.adsh=Ndebt.adsh and Ndebt.ddate=Nrevenue.ddate
      join sec_dera.num as Ncash on S.adsh=Ncash.adsh and Ncash.ddate=Nrevenue.ddate
      join sec_dera.num as Nfloat on S.adsh=Nfloat.adsh
where S.fp='FY' and (S.form='10-K' or S.form='20-F')
      -- and S.lei='I1BZKREC126H0VB1BL91'
and Nrevenue.coreg is NULL
and Nrevenue.uom='USD'
and (Nrevenue.tag='Revenues'
     or Nrevenue.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
     or Nrevenue.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
     or Nrevenue.tag='RevenuesNetOfInterestExpense'
     or Nrevenue.tag='RegulatedAndUnregulatedOperatingRevenue'
     or Nrevenue.tag='RegulatedOperatingRevenuePipelines')
and Nfloat.qtrs=0 and Nfloat.ddate>=fy and date_add('year', 1, fy)>Nfloat.ddate
and Nfloat.coreg is NULL
and Nfloat.uom='USD'
and (Nfloat.tag='EntityPublicFloat'
     or Nfloat.tag='EntitysPublicFloat'
     or Nfloat.tag='FreeFloat'
     or Nfloat.tag='PublicFloat'
     or Nfloat.tag='PublicFloatValue')
and Ncash.qtrs=0
and Ncash.coreg is NULL
and Ncash.uom='USD'
and (Ncash.tag='CashAndCashEquivalentsAtCarryingValue'
     or Ncash.tag='Cash'
     or Ncash.tag='CashEquivalentsAtCarryingValue'
     or Ncash.tag='CashAndCashEquivalents')
and Ndebt.qtrs=0
and Ndebt.coreg is NULL
and Ndebt.uom='USD'
and (Ndebt.tag='LongTermDebt'
     or Ndebt.tag='LongTermDebtFairValue'
     or Ndebt.tag='LongTermDebtAndCapitalLeaseObligations'
     or Ndebt.tag='DebtLongtermAndShorttermCombinedAmount'
     or Ndebt.tag='SecuredDebt'
     or Ndebt.tag='UnsecuredDebt'
     or Ndebt.tag='SubordinatedDebt'
     or Ndebt.tag='ConvertibleDebt')
and Nassets.qtrs=0
and Nassets.coreg is NULL
and Nassets.uom='USD'
and Nassets.tag='Assets'
group by name, lei, fy, Nrevenue.ddate, sic
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.financials_by_lei
""")
l = qres.fetchall()

print(len(l))

[(True,)]

TrinoQueryError: TrinoQueryError(type=INTERNAL_ERROR, name=PAGE_TRANSPORT_TIMEOUT, message="Encountered too many errors talking to a worker node. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (http://10.129.2.18:8080/v1/task/20211213_150737_00037_85hmd.2.2/results/0/0 - 100 failures, failure duration 300.40s, total failed request time 302.30s)", query_id=20211213_150737_00037_85hmd)

In [ ]:
columns = [x[0] for x in engine.execute("describe sec_dera.financials_by_lei").fetchall()]

In [ ]:
df = pd.DataFrame(data=l, columns=columns)

In [ ]:
df

In [ ]:
df = df.convert_dtypes()

drop_unmanaged_table("osc_datacommons_dev", "sec_dera", "corp_data_df", engine, trino_bucket, verbose=True)

drop_unmanaged_data("sec_dera", "corp_data_df", trino_bucket, verbose=True)

ingest_unmanaged_parquet(df, "sec_dera", "corp_data_df", trino_bucket, partition_columns=[],
                         append=False, workdir='/tmp', verbose=True)

sql = unmanaged_parquet_tabledef(df, "osc_datacommons_dev", "sec_dera", "corp_data_df", trino_bucket, partition_columns = [],
                                 verbose=True)
qres = engine.execute(sql)
display(qres.fetchall())

qres = engine.execute("select * from sec_dera.corp_data_df")
l = qres.fetchall()

In [ ]:
qres = engine.execute("""
explain select * from sec_dera.financials_by_lei
""")
print(qres.fetchall()[0][0])